In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import urllib

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import keras

Using TensorFlow backend.


In [6]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [7]:
from sklearn.cluster import KMeans

In [8]:
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

In [9]:
import requests
live_api_url = 'http://challenges.instagram.unpossib.ly/api/live'

In [10]:
import json

In [11]:
import time

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [13]:
import datetime

In [14]:
model = VGG16(weights='imagenet', include_top=False)

In [29]:
data = pd.read_csv('data.csv', index_col = 'Unnamed: 0').drop([1901,1989,2004,2011,2012,3039,4086,4990]).sample(frac=1)

In [30]:
X = data.drop(['thumbnail_src','comments','likes'], axis=1).reset_index(drop=True)

In [31]:
y = data['likes'].reset_index(drop=True)

In [32]:
X_img = pd.read_csv('insta_image_features.csv', index_col = 'Unnamed: 0')

In [33]:
X_img = X_img.loc[data.index.values].reset_index(drop=True)

In [34]:
Clust = KMeans(n_clusters = 50, random_state = 241)

In [35]:
Clust.fit(X_img)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=50, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=241, tol=0.0001,
    verbose=0)

In [36]:
X['Labels50'] = Clust.labels_

In [37]:
X

,adult,blue,confidence,green,id,medical,piFra,red,score,score_d,spoof,violence,description,Labels50
0,1,154,0.8,122,5136815,1,0.054631,80,0.293598,0.823548,1,1,190,36
1,1,27,0.8,48,5136815,1,0.047556,62,0.131204,0.968864,1,1,76,2
2,1,53,0.8,89,276192188,1,0.053611,71,0.103867,0.907313,1,1,168,22
3,1,28,0.8,26,364395,1,0.239289,22,0.239888,0.930194,1,1,36,1
4,1,138,0.8,88,5136815,1,0.094667,26,0.358171,0.839848,1,1,198,9
5,1,78,1.0,51,356564939,1,0.194800,41,0.261974,0.968864,1,1,76,0
6,1,195,0.8,193,364395,1,0.150400,190,0.365406,0.929066,1,3,36,9
7,1,6,0.8,68,364395,1,0.018844,192,0.224112,0.970095,1,1,258,1
8,1,23,0.8,54,356564939,1,0.011775,136,0.148506,0.795672,1,2,110,38
9,1,56,0.8,84,5136815,1,0.070333,83,0.238139,0.951600,1,1,191,11


In [38]:
X = X.drop(['score_d', 'description'], axis=1)

In [39]:
grid = {'max_depth':[3]}
cv = KFold(len(X), n_folds=5, shuffle=True, random_state=241)
clf = xgb.XGBRegressor()
gs = GridSearchCV(clf, grid, scoring='mean_absolute_error', cv=cv)

In [40]:
gs.fit(X.values,y.values)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=5024, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=100,
       nthread=-1, objective='reg:linear', seed=0, silent=True,
       subsample=1),
       fit_params={}, iid=True, n_jobs=1, param_grid={'max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_absolute_error',
       verbose=0)

In [41]:
gs.best_score_

-7833.6470660580808

In [42]:
id_predicted = []

### Live predict

In [ ]:
frame_empti = 1
while True:
    while frame_empti == 1:
        
        time.sleep(6) # delays for 5 seconds
        
        resp = requests.get(live_api_url, auth=('mp3shin@gmail.com', 'gXRuxMx54Kex89XRfZlNetMrQ3JD6tTX')).json()
        Frame_1 = pd.DataFrame(resp['accounts'])
        Frame_1['chek'] = 0
        for i in np.arange(0,5,1):
            Frame_1.set_value(i, 'chek', 1) if Frame_1.ix[i]['posts'] != [] else 0
        Frame_1 = Frame_1[Frame_1['chek']==1].drop(['chek'],axis=1).reset_index(drop=True)
    
        if Frame_1.empty:
            frame_empti = 1
        else:
            ap = []
            for i in np.arange(0,len(Frame_1),1):
                for k in np.arange(0,len(pd.DataFrame.from_dict(Frame_1['posts'].ix[i])),1):
                    sl_ = {'id':Frame_1.ix[i]['id'],
                           'confidence':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['cropHintsAnnotation']['cropHints'][0]['confidence'],
                           #'imFra':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['cropHintsAnnotation']['cropHints'][0]['importanceFraction'],
                           'blue':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue'],
                           'green':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green'],
                           'red':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red'],
                           'piFra':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction'],
                           'score':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score'],
                           #'descrip':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['labelAnnotations'][0]['description'],
                           #'score':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['labelAnnotations'][0]['score'],
                           'adult':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['safeSearchAnnotation']['adult'],
                           'medical':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['safeSearchAnnotation']['medical'],
                           'spoof':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['safeSearchAnnotation']['spoof'],
                           'violence':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['annotations'].ix[k]['safeSearchAnnotation']['violence'],
                           'comments':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['comments']['count'],
                           #'comments_disabled':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['comments_disabled'],
                           #'is_video':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['is_video'],
                           'likes':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['likes']['count'],
                           'thumbnail_src':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['thumbnail_src'],
                           'post_id':pd.DataFrame.from_dict(Frame_1['posts'].ix[i])['instagram'].ix[k]['id'], 
                          }
            
                    ap.append(sl_)
            data = pd.DataFrame(ap)
                       
            data['adult'][data['adult'] == 'VERY_UNLIKELY'] = 1
            data['adult'][data['adult'] == 'UNLIKELY'] = 2
            data['adult'][data['adult'] == 'POSSIBLE'] = 3
            data['adult'][data['adult'] == 'LIKELY'] = 4
            data['adult'][data['adult'] == 'VERY_LIKELY'] = 5

            data['medical'][data['medical'] == 'VERY_UNLIKELY'] = 1
            data['medical'][data['medical'] == 'UNLIKELY'] = 2
            data['medical'][data['medical'] == 'POSSIBLE'] = 3
            data['medical'][data['medical'] == 'LIKELY'] = 4
            data['medical'][data['medical'] == 'VERY_LIKELY'] = 5

            data['spoof'][data['spoof'] == 'VERY_UNLIKELY'] = 1
            data['spoof'][data['spoof'] == 'UNLIKELY'] = 2
            data['spoof'][data['spoof'] == 'POSSIBLE'] = 3
            data['spoof'][data['spoof'] == 'LIKELY'] = 4
            data['spoof'][data['spoof'] == 'VERY_LIKELY'] = 5

            data['violence'][data['violence'] == 'VERY_UNLIKELY'] = 1
            data['violence'][data['violence'] == 'UNLIKELY'] = 2
            data['violence'][data['violence'] == 'POSSIBLE'] = 3
            data['violence'][data['violence'] == 'LIKELY'] = 4
            data['violence'][data['violence'] == 'VERY_LIKELY'] = 5
            
            for i in np.arange(0,len(data),1):
                urllib.request.urlretrieve(data['thumbnail_src'].ix[i], str(data['post_id'].ix[i]) + '.jpg')
            idx = data.index.values
            features = []
            
            for i in idx:
                img_path = str(data['post_id'].ix[i])+'.jpg'
                img = image.load_img(img_path, target_size=(224, 224))
    
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
    
                features.append(model.predict(x).flatten())
        
            X_img_live = pd.DataFrame(features, index=idx)
            data['Labels50'] = Clust.predict(X_img_live)
            X_live = data.drop(['thumbnail_src','comments','likes','post_id'], axis=1)
            Frame_f = pd.concat([pd.DataFrame(gs.predict(X_live.values)), data[['post_id']]], axis=1)
            
            for i in Frame_f.index.values:
                submit = json.dumps({'post':Frame_f.ix[i]['post_id'], 'likes':int(Frame_f.ix[i][0])}, ensure_ascii=False)
                
                if Frame_f.ix[i]['post_id'] not in id_predicted:
                    requests.post("http://challenges.instagram.unpossib.ly/api/submissions",
                                  auth=('mp3shin@gmail.com', 'gXRuxMx54Kex89XRfZlNetMrQ3JD6tTX'),
                                  data=submit)
                    
                    id_predicted.append(Frame_f.ix[i]['post_id'])